<a href="https://colab.research.google.com/github/adityakalkeri1/Projects/blob/NLP/Sentiment140_using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!ls '/content/gdrive/MyDrive/Datasets/'

archive.zip


In [ ]:
!unzip 'gdrive/MyDrive/Datasets/archive.zip' -d 'Archive/'

Archive:  gdrive/MyDrive/Datasets/archive.zip
  inflating: Archive/training.1600000.processed.noemoticon.csv  


In [ ]:
df = pd.read_csv('Archive/training.1600000.processed.noemoticon.csv', encoding = 'latin', header=None)

In [ ]:
df.columns = ['Sentiment', 'ID', 'Date', 'Query', 'user', 'Text']

In [ ]:
df.drop(['ID', 'Date', 'Query', 'user'], axis =1, inplace = True)

In [ ]:
def text_cleaner(row):
  row = re.sub('@[a-zA-Z0-9_]+','', row)                                          #For removing usernames
  row = re.sub('(http.+\/\/[a-zA-Z.]+\/\S+|http.+\/\/[a-zA-Z0-9.]+\/)', '', row)  #For removing website names
  row = re.sub('[0-9]+', '', row)                                                 #For removing numbers
  row = re.sub('[*\(\)\?!_%-]', '', row)                                           #For removing symbols

  return row
df['Text'] = df['Text'].apply(text_cleaner)

In [ ]:
label = df['Sentiment'].values
label[label == 4] = 1
text = [x for x in df['Text'].values]

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 51kB 4.0MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=78248e04e6ca630da774b67a76413b93d8ecbd693fc8cee08696f581dcf3bc37
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=d673d9c09838e35b2e0dd8ec39025e3934e2b04467e27c15abd24c3fbd53f2ce
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=21b35c7bd18f6e365637fb94fbcc0adafafd9a87f03b728e9d218fea2a9d32bd
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.2MB 5.5MB/s 


In [ ]:
import bert
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
#Encoding sentence with tokens
def encode_sentence(sent):
    return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [ ]:
data_inputs = [encode_sentence(sentence) for sentence in text]

In [ ]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id                                    # turns 1 into 0 and vice versa
    return seg_ids

In [ ]:
import random
len_of_sent = [[sent, label[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(len_of_sent)
len_of_sent.sort(key=lambda x: x[2])
dataset_sorted = [([get_ids(sent_lab[0]),
                get_mask(sent_lab[0]),
                get_segments(sent_lab[0])],
               sent_lab[1])
              for sent_lab in len_of_sent if sent_lab[2] > 7]

In [ ]:
# A list is a type of iterator so it can be used as generator for a dataset using lambda function
all_dataset = tf.data.Dataset.from_generator(lambda: dataset_sorter,                            
                                             output_types=(tf.int32, tf.int32))

In [ ]:
#Here we prepare padded batches
BATCH_SIZE = 64
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0))

In [ ]:
import math
NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [ ]:
from tensorflow.keras import layers
class CNN_embedding(tf.keras.Model):
    
    def __init__(self,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 name="dcnn"):
        super(CNN_embedding, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=False)

        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def embed_with_bert(self, all_tokens):
        _, embs = self.bert_layer([all_tokens[:, 0, :],
                                   all_tokens[:, 1, :],
                                   all_tokens[:, 2, :]])
        return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)

        print(x.shape)

        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

In [ ]:
NB_FILTERS = 100
FFN_UNITS = 512
NB_CLASSES = 2

DROPOUT_RATE = 0.1

BATCH_SIZE = 64
NB_EPOCHS = 1

In [ ]:
model = CNN_embedding(nb_filters=NB_FILTERS,
                         FFN_units=FFN_UNITS,
                         nb_classes=NB_CLASSES,
                         dropout_rate=DROPOUT_RATE)

In [ ]:
model.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])

In [ ]:
model.fit(train_dataset,
         epochs=NB_EPOCHS,
         )

(None, None, 768)
(None, None, 768)
20101/20101 [==============================] - 4004s 196ms/step - loss: 0.3881 - accuracy: 0.8271


In [ ]:
model.save('/content/gdrive/MyDrive/Deep learning model/bert', save_format='tf')

(None, None, 768)
(None, None, 768)
(None, None, 768)
(None, None, 768)
(None, None, 768)
(None, None, 768)
(None, None, 768)


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Deep learning model/bert/assets


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Deep learning model/bert/assets


In [ ]:
model.evaluate(test_dataset)

(None, None, 768)
2233/2233 [==============================] - 207s 92ms/step - loss: 0.3638 - accuracy: 0.8406


[0.3638046681880951, 0.8406081795692444]